![Imandrabot](../docs/images/kostya_ros_medium_1.png)

# 1. ROS message OCaml types

For our Imandra-ROS project we’ve processed all the standard ROS messages with our code generation tool creating a collection of strongly-typed IML/OCaml bindings for them. But, in order to keep this notebook self-contained we'll define the necessary messaging modules here.  

First, we'll need to declare the message type that will control our robot. This is typically done with a `Twist` message from the `geometry_msgs` standard ROS package. We want to mimic ROS messaging nomenclauture as close as possible, so we'll create an OCaml/Imadra `module` with the same name as the package and will place the necessary type/message declaraions inside:

In [1]:
module Geometry_msgs = struct
  type vector3 = 
    { vector3_x : int
    ; vector3_y : int
    ; vector3_z : int
    }
  type twist = 
    { twist_linear  : vector3
    ; twist_angular : vector3
    }    
end

module Geometry_msgs :
  sig
    type vector3 = { vector3_x : int; vector3_y : int; vector3_z : int; }
    type twist = { twist_linear : vector3; twist_angular : vector3; }
  end


One thing that you might have noticed is that we've replaced floating point values for vector coordinates with `int`s. It is much easier for Imandra to reason about integers, so we assume that there is a common factor of 100000 that multiplying all the incoming floating point values and divides all the outgoing integers. (That effectively makes our unit of measurement to be 10 micrometers).

Let's move on and declare the incoming messages: 
 - `LaserScan` sensor input message from the `sensor_msgs` ROS package
 - and the `Clock` message from the `Rosgraph_msg` ROS package  
 
We define the wrapping modules for both messages and declare their datatypes:

In [2]:
module Sensor_msgs = struct
  type laserScan = 
    { laserScan_range_min : int 
    ; laserScan_range_max : int 
    ; laserScan_ranges : int list 
    }
end
module Rosgraph_msgs = struct
  type time = 
    { seconds     : int
    ; nanoseconds : int
    }
  type clock = { clock : time }
end

module Sensor_msgs :
  sig
    type laserScan = {
      laserScan_range_min : int;
      laserScan_range_max : int;
      laserScan_ranges : int list;
    }
  end
module Rosgraph_msgs :
  sig
    type time = { seconds : int; nanoseconds : int; }
    type clock = { clock : time; }
  end


# 2. Creating a simple ROS Node model

## 2.1 State datatype

Working with Imandra we’ve adopted a standard way to construct formal models of message-driven systems. At the top of the model we have a single OCaml datatype that holds all the data needed to describe the system at a given moment, including incoming and outgoing messages. We call this record type `state`. Together with this `state` type we define a `one_step` transition `state -> state` function, which performs a single logically isolated step of the simulation and returns the new `state` after the transition.

As an example, consider an IML/OCaml type declaration for a simple ROS node that is able to accept `rosgraph_msgs/Clock` and `sensor_msgs/LaserScan` standard ROS messages. We also want the state to store two values: 
 - the latest minimal value of the ranges that the laser sensor returns
 - the preferred side for the robot to turn -- either left or right. 
 
Finally, we want the node to be able to send `geometry_msgs/Twist` ROS message depending on the stored `min_range` data:

In [3]:
type incoming_msg = 
  | Clock  of Rosgraph_msgs.clock
  | Sensor of Sensor_msgs.laserScan

type outgoing_msg =
  | Twist of Geometry_msgs.twist

type direction = CW | CCW 
type mode = Driving | Turning 

type state =
  { mode : mode
  ; min_range : int option
  ; direction : direction option
  ; incoming  : incoming_msg option
  ; outgoing  : outgoing_msg option 
  }

type incoming_msg =
    Clock of Rosgraph_msgs.clock
  | Sensor of Sensor_msgs.laserScan
type outgoing_msg = Twist of Geometry_msgs.twist
type direction = CW | CCW
type mode = Driving | Turning
type state = {
  mode : mode;
  min_range : int option;
  direction : direction option;
  incoming : incoming_msg option;
  outgoing : outgoing_msg option;
}


## 2.2 State transition `one_step` function

In [4]:
let fold_left_i f acc lst = 
  let rec scan lst acc i = 
    match lst with
    | h::tl -> scan tl (f h acc i) (i+1) 
    | [] -> acc
    in 
  scan lst acc 0

val fold_left_i : ('a -> 'b -> Z.t -> 'b) -> 'b -> 'a list -> 'b = <fun>


termination proof Termination proof call `recfun.fold_left_i.scan.1 f_0 (List.tl lst) (f_0 (List.hd lst) acc i) (i + 1)` from `recfun.fold_left_i.scan.1 f_0 lst acc i` original recfun.fold_left_i.scan.1 f_0 lst acc i sub recfun.fold_left_i.scan.1 f_0 (List.tl lst) (f_0 (List.hd lst) acc i) (i + 1) original ordinal Ordinal.Int (Ordinal.count lst) sub ordinal Ordinal.Int (Ordinal.count (List.tl lst)) path [lst <> []] proof detailed proof summary full ground_instances 3 definitions 0 inductions 0 search_time 0.012s details Expand smt_stats (:added-eqs 61
 :arith-add-rows 21
 :arith-assert-diseq 1
 :arith-assert-lower 16
 :arith-assert-upper 14
 :arith-conflicts 1
 :arith-eq-adapter 15
 :arith-fixed-eqs 14
 :arith-pivots 8
 :conflicts 8
 :datatype-accessor-ax 9
 :datatype-constructor-ax 7
 :datatype-occurs-check 32
 :datatype-splits 1
 :decisions 16
 :del-clause 12
 :final-checks 4
 :max-memory 18.35
 :memory 15.95
 :mk-bool-var 82
 :mk-clause 22
 :num-allocs 491445138
 :num-checks 8
 :propagations 19
 :rlimit-count 67444) Expand start[0.012s]
 lst <> [] && Ordinal.count lst >= 0 && Ordinal.count (List.tl lst) >= 0
 ==> not ((List.tl lst) <> [])
 || Ordinal.Int (Ordinal.count (List.tl lst)) Ordinal.<<
 Ordinal.Int (Ordinal.count lst) simplify into (not
 ((lst <> [] && Ordinal.count lst >= 0) && Ordinal.count (List.tl lst) >= 0)
 || not ((List.tl lst) <> []))
|| Ordinal.Int (Ordinal.count (List.tl lst)) Ordinal.<<
 Ordinal.Int (Ordinal.count lst) expansions [] rewrite_steps forward_chaining unroll expr (Ordinal.<<_130 (Ordinal.Int_121 (|count_`ty_1 list`_2153|
 (|get.… expansions unroll expr (|count_`ty_1 list`_2153| (|get.::.1_2139| lst_2146)) expansions unroll expr (|count_`ty_1 list`_2153| lst_2146) expansions unsat (let ((a!1 (or (= lst_2146
 (|::_2| (|get.::.0_2138| lst_2146) (|get.::.1_2139| lst…

Lets now create a simple `one_step` function for a ROS node that reacts to the incoming messages. We want the robot, on incoming `Clock` message, to either move forward or turn. On incoming `Scan` message, 


depending on the stored `min_range` value. To this end, we first create two helper functions: 
 - `get_min_range` extract the minimal value of the ranges in a `Scan` message 
 - `make_twist_message` creates a `Twist` message with the given forward and angular velocities

In [5]:
let get_min_and_direction msg =
  let max = msg.Sensor_msgs.laserScan_range_max in
  let lst = msg.Sensor_msgs.laserScan_ranges in
  let min = fold_left_i 
    (fun a b i -> if a < fst b then (a,i) else b) (max, 0) in
  let min_range, idx = min lst in
  if idx < List.length lst / 2 then min_range, CW else min_range, CCW
  
let make_twist_message v omega=
  let open Geometry_msgs in
  let mkvector x y z = { vector3_x = x; vector3_y = y; vector3_z = z } in 
  Twist { twist_linear  = mkvector v 0 0
        ; twist_angular = mkvector 0 0 omega 
        } 

val get_min_and_direction : Sensor_msgs.laserScan -> int * direction = <fun>
val make_twist_message : int -> int -> outgoing_msg = <fun>


In [6]:
let process_clock_message state =
  match state.mode with 
  | Driving -> { state with outgoing = Some (make_twist_message 10000 0) } 
  | Turning -> begin
    match state.direction with 
    | None
    | Some ( CW ) -> { state with outgoing = Some (make_twist_message 0   10000) } 
    | Some (CCW ) -> { state with outgoing = Some (make_twist_message 0 (-10000))} 
  end

val process_clock_message : state -> state = <fun>


In [7]:
let process_sensor_message state min_range min_direction =
  match state.mode , state.min_range with 
  | Driving , _    ->
    if min_range < 20000 then 
      { state with 
        mode      = Turning
      ; direction = Some min_direction
      ; min_range = Some min_range 
      }
    else
      { state with mode = Driving; min_range = None; direction = None } 
  | Turning , None -> 
    if min_range > 25000 then 
      { state with mode = Driving; min_range = None; direction = None } 
    else
      { state with 
        direction = Some min_direction
      ; min_range = Some min_range
      } 
  | Turning , Some old_range -> 
    if min_range > 25000 then 
      { state with mode = Driving; min_range = None; direction = None } 
    else if min_range > old_range then state else 
    { state with 
      direction = Some min_direction
    ; min_range = Some min_range
    }  

val process_sensor_message : state -> int -> direction -> state = <fun>


With the help of these functions, we can create our `one_step` transition function:

In [8]:
let one_step state =
  match state.incoming with None -> state | Some in_msg ->
  let state = { state with incoming = None; outgoing = None } in
  match in_msg with 
  | Sensor laserScan -> 
    let min_range, min_direction = get_min_and_direction laserScan in
    process_sensor_message state min_range min_direction
  | Clock  _ -> 
    process_clock_message state

val one_step : state -> state = <fun>


## 2.3 Running the model as a ROS node

One can write the model into an OCaml code file:   
https://github.com/AestheticIntegration/imandra-ros/blob/master/imandra_model/src-model/ros_model.ml  
and then compile it into an executable node, using our ROS node wrapper. 

Here is the model, controling our "imandrabot" in the Gazebo simulation environment:

dd

# 3. Verifying the ROS node model

## 3.1 Verifying outgoing `Twist` message at `Clock` ticks 

Our model is designed in such a way that it updates its state parameters upon `LaserScan` messages 


$$ \forall s. IsClock(IncomingMessage(s)) \,\Rightarrow\, IsTwist(OutgoingMessage(s)) $$

Meaning that for every state $s$, if the state contains an incoming message and this message is a `Clock` message, then the state's `outgoing` message is a `Twist` after we've called `one_step` on it.  

We can almost literally encode this formal expression as an Imandra `theorem`:

In [13]:
let is_clock msg = match msg with  Some ( Clock _ ) -> true | _ -> false ;;
let is_twist msg = match msg with  Some ( Twist _ ) -> true | _ -> false ;;

theorem clock_creates_outbound state =
  is_clock state.incoming ==> is_twist (one_step state).outgoing

val is_clock : incoming_msg option -> bool = <fun>
val is_twist : outgoing_msg option -> bool = <fun>
val clock_creates_outbound : state -> bool = <fun>


Proved proof call graph proof summary full graph ground_instances 0 definitions 0 inductions 0 search_time 0.006s details Expand smt_stats (:added-eqs 293
 :arith-assert-lower 2
 :arith-assert-upper 2
 :arith-eq-adapter 1
 :conflicts 10
 :datatype-accessor-ax 48
 :datatype-constructor-ax 33
 :datatype-splits 28
 :decisions 17
 :del-clause 3
 :max-memory 18.35
 :memory 13.49
 :mk-bool-var 215
 :mk-clause 39
 :num-allocs 621003603
 :num-checks 2
 :propagations 35
 :rlimit-count 75121) Expand start[0.006s]
 (if Is_a(Clock, Option.get :var_0:.incoming)
 && Is_a(Some, :var_0:.incoming)
 then true else false)
 ==> (if Is_a(Twist,
 Option.get
 (if :var_0:.incoming = None then :var_0: else …).outgoing)
 && Is_a(Some,
 (if :var_0:.incoming = None then :var_0: else …).outgoing)
 then true else false) simplify into not
(Is_a(Clock, Option.get :var_0:.incoming) && Is_a(Some, :var_0:.incoming))
|| Is_a(Twist,
 Option.get (if :var_0:.incoming = None then :var_0: else …).outgoing)
 && Is_a(Some, (if :var_0:.incoming = None then :var_0: else …).outgoing) expansions [] rewrite_steps forward_chaining unsat (let ((a!1 (Sensor_msgs.laserScan_ranges_2078
 (get.Sensor.0_2093 (get.Some.0_2113 (inco… digraph "proof" {
p_213 [label="Start ((if Is_a(Clock, Option.get :var_0:.incoming)\l && Is_a(Some, :var_0:.incoming)\l then true else false)\l ==\> (if Is_a(Twist,\l Option.get\l (if :var_0:.incoming = None then :var_0: else …).outgoing)\l && Is_a(Some,\l (if :var_0:.incoming = None then :var_0: else …).outgoing)\l then true else false)\l :time 0.006s)",shape=box,style=filled,fontname="courier",fontsize=14];
p_213 -> p_212 [label=""];
p_212 [label="Simplify (not\l (Is_a(Clock, Option.get :var_0:.incoming)\l && Is_a(Some, :var_0:.incoming))\l \|\| Is_a(Twist,\l Option.get\l (if :var_0:.incoming = None then :var_0: else …).outgoing)\l && Is_a(Some,\l (if :var_0:.incoming = None then :var_0: else …).outgoing)\l :expansions [] :rw [] :fc [])",shape=box,style=filled,fontname="courier",fontsize=14];
p_212 -> p_211 [label=""];
p_211 -> p_z3_3094 [label="",fontname="courier",fontsize=14];
p_z3_3094 [label="(let ((a!1 (Sensor_msgs.laserScan_ranges_2078
 (get.Sensor.0_2093 (get.Some.0_2113 (inco…",shape=box,fontname="courier",fontsize=14];p_211 [label="unsat",shape=box,style=filled,fontname="courier",fontsize=14];
}
 Load graph Loading.. digraph "call graph" {
goal [label="not\l(not (Is_a(Clock, Option.get state.incoming) && Is_a(Some, state.incoming))\l \|\| Is_a(Twist,\l Option.get (if state.incoming = None then state else …).outgoing)\l && Is_a(Some, (if state.incoming = None then state else …).outgoing))",shape=box,style=filled,color="cyan",fontname="courier",fontsize=14];
call_1379 [label="List.length\l(Destruct(Sensor, 0, Option.get …)).Sensor_msgs.laserScan_ranges",shape=box,style=filled,color="yellow",fontname="courier",fontsize=14];
goal -> call_1379 [label="calls"];
call_1895 [label="recfun.fold_left_i.scan.1 anon_fun.get_min_and_direction.0\l(Destruct(Sensor, 0, Option.get …)).Sensor_msgs.laserScan_ranges\l((Destruct(Sensor, 0, Option.get …)).Sensor_msgs.laserScan_range_max, 0) 0",shape=box,style=filled,color="yellow",fontname="courier",fontsize=14];
goal -> call_1895 [label="calls"];
}
 Load graph Loading..

One can see that the theorem is "Proven", meaning that Imandra has formally checked that this property holds for all possible input states. 


## 3.2 Veryfying no dirving if close to a wall

## 3.2 Verifying initialization `min_range` value

The `min_range` state variable is an `option` and initially is `None`. Lets verify that upon receiving a `sensor_msgs/LaserScan` message, the resulting state stores some `min_range` value (not `None`). 

$$ \forall s. IsLaserScan(IncomingMessage(s)) \,\Rightarrow\, MinValue(one\_step(s)) \ne None $$

Meaning that for every state $s$, if the state contains an incoming message and this message is `LaserScan`, then the state's `min_value` is not `None` after we've called `one_step` on it.  

Again, we encode this formal expression as an Imandra `theorem`:

In [16]:
let is_laser_scan msg = 
  match msg with  Some ( Sensor _ ) -> true | _ -> false ;;
  
verify (fun state ->
  is_laser_scan state.incoming ==> (one_step state).min_range <> None)

val is_laser_scan : incoming_msg option -> bool = <fun>
- : state -> bool = <fun>
module CX : sig val state : state end


Refuted call graph proof-attempt digraph "call graph" {
goal [label="not\l(not (Is_a(Sensor, Option.get state.incoming) && Is_a(Some, state.incoming))\l \|\| not\l ((if state.incoming = None then state\l else\l if Is_a(Sensor, Option.get state.incoming)\l then if state.mode = Driving then … else …\l else if state.mode = Driving then … else …).min_range\l = None))",shape=box,style=filled,color="cyan",fontname="courier",fontsize=14];
call_3701 [label="List.length\l(Destruct(Sensor, 0, Option.get …)).Sensor_msgs.laserScan_ranges",shape=box,style=filled,color="green",fontname="courier",fontsize=14];
call_3701 -> expr_4445 [label="="];
expr_4445 [label="if (Destruct(Sensor, 0, Option.get ….incoming)).Sensor_msgs.laserScan_ranges\l = []\lthen 0\lelse\l 1\l + List.length\l (List.tl\l (Destruct(Sensor, 0, Option.get ….incoming)).Sensor_msgs.laserScan_ranges)",shape=box,style=filled,color="grey",fontname="courier",fontsize=14];
goal -> call_3701 [label="calls"];
call_4461 [label="recfun.fold_left_i.scan.1 anon_fun.get_min_and_direction.0\l(List.tl (Destruct(Sensor, 0, Option.get …)).Sensor_msgs.laserScan_ranges)\l(if (Destruct(Sensor, 0, Option.get …)).Sensor_msgs.laserScan_range_max \<=\l List.hd\l (Destruct(Sensor, 0, Option.get …)).Sensor_msgs.laserScan_ranges\l then\l ((Destruct(Sensor, 0, Option.get …)).Sensor_msgs.laserScan_range_max, 0)\l else\l (List.hd\l (Destruct(Sensor, 0, Option.get …)).Sensor_msgs.laserScan_ranges, 0))\l(1 + 0)",shape=box,style=filled,color="yellow",fontname="courier",fontsize=14];
expr_4462 -> call_4461 [label="calls"];
call_3705 [label="recfun.fold_left_i.scan.1 anon_fun.get_min_and_direction.0\l(Destruct(Sensor, 0, Option.get …)).Sensor_msgs.laserScan_ranges\l((Destruct(Sensor, 0, Option.get …)).Sensor_msgs.laserScan_range_max, 0) 0",shape=box,style=filled,color="green",fontname="courier",fontsize=14];
call_3705 -> expr_4462 [label="="];
expr_4462 [label="if (Destruct(Sensor, 0, Option.get …)).Sensor_msgs.laserScan_ranges \<\> []\lthen\l recfun.fold_left_i.scan.1 anon_fun.get_min_and_direction.0\l (List.tl (Destruct(Sensor, 0, Option.get …)).Sensor_msgs.laserScan_ranges)\l (if … \<=\l List.hd\l (Destruct(Sensor, 0, Option.get …)).Sensor_msgs.laserScan_ranges\l then (…, 0)\l else\l (List.hd\l (Destruct(Sensor, 0, Option.get …)).Sensor_msgs.laserScan_ranges, 0))\l (1 + 0)\lelse (…, 0)",shape=box,style=filled,color="grey",fontname="courier",fontsize=14];
goal -> call_3705 [label="calls"];
call_4448 [label="List.length\l(List.tl (Destruct(Sensor, 0, Option.get …)).Sensor_msgs.laserScan_ranges)",shape=box,style=filled,color="yellow",fontname="courier",fontsize=14];
expr_4445 -> call_4448 [label="calls"];
}
 Load graph Loading.. proof attempt summary full graph ground_instances 2 definitions 0 inductions 0 search_time 0.014s details Expand smt_stats (:added-eqs 779
 :arith-add-rows 4
 :arith-assert-lower 13
 :arith-assert-upper 14
 :arith-eq-adapter 6
 :arith-fixed-eqs 2
 :arith-offset-eqs 2
 :arith-pivots 5
 :conflicts 20
 :datatype-accessor-ax 44
 :datatype-constructor-ax 135
 :datatype-occurs-check 753
 :datatype-splits 32
 :decisions 140
 :del-clause 9
 :final-checks 6
 :max-memory 18.35
 :memory 16.27
 :minimized-lits 4
 :mk-bool-var 336
 :mk-clause 61
 :num-allocs 686212387
 :num-checks 5
 :propagations 205
 :rlimit-count 83362) Expand start[0.014s]
 (if Is_a(Sensor, Option.get :var_0:.incoming)
 && Is_a(Some, :var_0:.incoming)
 then true else false)
 ==> not
 ((if :var_0:.incoming = None then :var_0:
 else
 if Is_a(Sensor, Option.get :var_0:.incoming)
 then if ….mode = Driving then … else …
 else if ….mode = Driving then … else …).min_range
 = None) simplify into not
(Is_a(Sensor, Option.get :var_0:.incoming) && Is_a(Some, :var_0:.incoming))
|| not
 ((if :var_0:.incoming = None then :var_0:
 else
 if Is_a(Sensor, Option.get :var_0:.incoming)
 then if :var_0:.mode = Driving then … else …
 else if :var_0:.mode = Driving then … else …).min_range
 = None) expansions [] rewrite_steps forward_chaining unroll

In [30]:
  
verify (fun state ->
  (  is_laser_scan state.incoming 
  && state.mode = Turning
  ) ==> (one_step state).min_range <> None)

- : state -> bool = <fun>
module CX : sig val state : state end


Refuted call graph proof-attempt digraph "call graph" {
goal [label="not\l(not\l ((Is_a(Sensor, Option.get state.incoming) && Is_a(Some, state.incoming))\l && state.mode = Turning)\l \|\| not\l ((if state.incoming = None then state\l else\l if Is_a(Sensor, Option.get state.incoming)\l then if state.mode = Driving then … else …\l else if state.mode = Driving then … else …).min_range\l = None))",shape=box,style=filled,color="cyan",fontname="courier",fontsize=14];
call_6014 [label="List.length\l(List.tl (Destruct(Sensor, 0, Option.get …)).Sensor_msgs.laserScan_ranges)",shape=box,style=filled,color="yellow",fontname="courier",fontsize=14];
expr_6009 -> call_6014 [label="calls"];
call_5316 [label="List.length\l(Destruct(Sensor, 0, Option.get …)).Sensor_msgs.laserScan_ranges",shape=box,style=filled,color="green",fontname="courier",fontsize=14];
call_5316 -> expr_6009 [label="="];
expr_6009 [label="if (Destruct(Sensor, 0, Option.get ….incoming)).Sensor_msgs.laserScan_ranges\l = []\lthen 0\lelse\l 1\l + List.length\l (List.tl\l (Destruct(Sensor, 0, Option.get ….incoming)).Sensor_msgs.laserScan_ranges)",shape=box,style=filled,color="grey",fontname="courier",fontsize=14];
goal -> call_5316 [label="calls"];
call_6026 [label="recfun.fold_left_i.scan.1 anon_fun.get_min_and_direction.0\l(List.tl (Destruct(Sensor, 0, Option.get …)).Sensor_msgs.laserScan_ranges)\l(if (Destruct(Sensor, 0, Option.get …)).Sensor_msgs.laserScan_range_max \<=\l List.hd\l (Destruct(Sensor, 0, Option.get …)).Sensor_msgs.laserScan_ranges\l then\l ((Destruct(Sensor, 0, Option.get …)).Sensor_msgs.laserScan_range_max, 0)\l else\l (List.hd\l (Destruct(Sensor, 0, Option.get …)).Sensor_msgs.laserScan_ranges, 0))\l(1 + 0)",shape=box,style=filled,color="yellow",fontname="courier",fontsize=14];
expr_6027 -> call_6026 [label="calls"];
call_5320 [label="recfun.fold_left_i.scan.1 anon_fun.get_min_and_direction.0\l(Destruct(Sensor, 0, Option.get …)).Sensor_msgs.laserScan_ranges\l((Destruct(Sensor, 0, Option.get …)).Sensor_msgs.laserScan_range_max, 0) 0",shape=box,style=filled,color="green",fontname="courier",fontsize=14];
call_5320 -> expr_6027 [label="="];
expr_6027 [label="if (Destruct(Sensor, 0, Option.get …)).Sensor_msgs.laserScan_ranges \<\> []\lthen\l recfun.fold_left_i.scan.1 anon_fun.get_min_and_direction.0\l (List.tl (Destruct(Sensor, 0, Option.get …)).Sensor_msgs.laserScan_ranges)\l (if … \<=\l List.hd\l (Destruct(Sensor, 0, Option.get …)).Sensor_msgs.laserScan_ranges\l then (…, 0)\l else\l (List.hd\l (Destruct(Sensor, 0, Option.get …)).Sensor_msgs.laserScan_ranges, 0))\l (1 + 0)\lelse (…, 0)",shape=box,style=filled,color="grey",fontname="courier",fontsize=14];
goal -> call_5320 [label="calls"];
}
 Load graph Loading.. proof attempt summary full graph ground_instances 2 definitions 0 inductions 0 search_time 0.017s details Expand smt_stats (:added-eqs 640
 :arith-add-rows 4
 :arith-assert-lower 14
 :arith-assert-upper 13
 :arith-eq-adapter 6
 :arith-fixed-eqs 2
 :arith-offset-eqs 2
 :arith-pivots 5
 :conflicts 17
 :datatype-accessor-ax 44
 :datatype-constructor-ax 93
 :datatype-occurs-check 753
 :datatype-splits 32
 :decisions 155
 :del-clause 11
 :final-checks 6
 :max-memory 20.03
 :memory 17.54
 :minimized-lits 3
 :mk-bool-var 290
 :mk-clause 55
 :num-allocs 868792382
 :num-checks 5
 :propagations 149
 :rlimit-count 100919) Expand start[0.017s]
 (if Is_a(Sensor, Option.get :var_0:.incoming)
 && Is_a(Some, :var_0:.incoming)
 then true else false)
 && :var_0:.mode = Turning
 ==> not
 ((if :var_0:.incoming = None then :var_0:
 else
 if Is_a(Sensor, Option.get :var_0:.incoming)
 then if ….mode = Driving then … else …
 else if ….mode = Driving then … else …).min_range
 = None) simplify into not
((Is_a(Sensor, Option.get :var_0:.incoming) && Is_a(Some, :var_0:.incoming))
 && :var_0:.mode = Turning)
|| not
 ((if :var_0:.incoming = None then :var_0:
 else
 if Is_a(Sensor, Option.get :var_0:.incoming)
 then if :var_0:.mode = Driving then … else …
 else if :var_0:.mode = Drivin

## 3.3 Veryfying stopping if close to a wall

In [28]:
let is_laser_scan_close msg = 
  let open Sensor_msgs in
  match msg with  
    | Some ( Sensor data ) -> 
      List.for_all (fun x -> x < 25000) data.laserScan_ranges  
    | _ -> false ;;

val is_laser_scan_close : incoming_msg option -> bool = <fun>


In [29]:
verify ( fun state ->
  (  is_laser_scan_close state.incoming
  ) ==> (one_step state).mode = Turning
)

- : state -> bool = <fun>
module CX : sig val state : state end


Refuted call graph proof-attempt digraph "call graph" {
goal [label="not\l(not\l ((Is_a(Sensor, Option.get state.incoming) && Is_a(Some, state.incoming))\l && List.for_all anon_fun.is_laser_scan_close.0\l (Destruct(Sensor, 0, Option.get state.incoming)).Sensor_msgs.laserScan_ranges)\l \|\| (if state.incoming = None then state\l else\l if Is_a(Sensor, Option.get state.incoming)\l then if state.mode = Driving then … else …\l else if state.mode = Driving then … else …).mode\l = Turning)",shape=box,style=filled,color="cyan",fontname="courier",fontsize=14];
call_522 [label="List.length\l(Destruct(Sensor, 0, Option.get …)).Sensor_msgs.laserScan_ranges",shape=box,style=filled,color="green",fontname="courier",fontsize=14];
call_522 -> expr_5037 [label="="];
expr_5037 [label="if (Destruct(Sensor, 0, Option.get ….incoming)).Sensor_msgs.laserScan_ranges\l = []\lthen 0\lelse\l 1\l + List.length\l (List.tl\l (Destruct(Sensor, 0, Option.get ….incoming)).Sensor_msgs.laserScan_ranges)",shape=box,style=filled,color="grey",fontname="courier",fontsize=14];
goal -> call_522 [label="calls"];
call_5166 [label="List.for_all anon_fun.is_laser_scan_close.0\l(List.tl (Destruct(Sensor, 0, Option.get …)).Sensor_msgs.laserScan_ranges)",shape=box,style=filled,color="yellow",fontname="courier",fontsize=14];
expr_5169 -> call_5166 [label="calls"];
call_2925 [label="List.for_all anon_fun.is_laser_scan_close.0\l(Destruct(Sensor, 0, Option.get …)).Sensor_msgs.laserScan_ranges",shape=box,style=filled,color="green",fontname="courier",fontsize=14];
call_2925 -> expr_5169 [label="="];
expr_5169 [label="(Destruct(Sensor, 0, Option.get ….incoming)).Sensor_msgs.laserScan_ranges =\l[]\l\|\| not\l (25000 \<=\l List.hd\l (Destruct(Sensor, 0, Option.get ….incoming)).Sensor_msgs.laserScan_ranges)\l && List.for_all anon_fun.is_laser_scan_close.0\l (List.tl\l (Destruct(Sensor, 0, Option.get ….incoming)).Sensor_msgs.laserScan_ranges)",shape=box,style=filled,color="grey",fontname="courier",fontsize=14];
goal -> call_2925 [label="calls"];
call_5054 [label="recfun.fold_left_i.scan.1 anon_fun.get_min_and_direction.0\l(List.tl (Destruct(Sensor, 0, Option.get …)).Sensor_msgs.laserScan_ranges)\l(if (Destruct(Sensor, 0, Option.get …)).Sensor_msgs.laserScan_range_max \<=\l List.hd\l (Destruct(Sensor, 0, Option.get …)).Sensor_msgs.laserScan_ranges\l then\l ((Destruct(Sensor, 0, Option.get …)).Sensor_msgs.laserScan_range_max, 0)\l else\l (List.hd\l (Destruct(Sensor, 0, Option.get …)).Sensor_msgs.laserScan_ranges, 0))\l(1 + 0)",shape=box,style=filled,color="yellow",fontname="courier",fontsize=14];
expr_5055 -> call_5054 [label="calls"];
call_20 [label="recfun.fold_left_i.scan.1 anon_fun.get_min_and_direction.0\l(Destruct(Sensor, 0, Option.get …)).Sensor_msgs.laserScan_ranges\l((Destruct(Sensor, 0, Option.get …)).Sensor_msgs.laserScan_range_max, 0) 0",shape=box,style=filled,color="green",fontname="courier",fontsize=14];
call_20 -> expr_5055 [label="="];
expr_5055 [label="if (Destruct(Sensor, 0, Option.get …)).Sensor_msgs.laserScan_ranges \<\> []\lthen\l recfun.fold_left_i.scan.1 anon_fun.get_min_and_direction.0\l (List.tl (Destruct(Sensor, 0, Option.get …)).Sensor_msgs.laserScan_ranges)\l (if … \<=\l List.hd\l (Destruct(Sensor, 0, Option.get …)).Sensor_msgs.laserScan_ranges\l then (…, 0)\l else\l (List.hd\l (Destruct(Sensor, 0, Option.get …)).Sensor_msgs.laserScan_ranges, 0))\l (1 + 0)\lelse (…, 0)",shape=box,style=filled,color="grey",fontname="courier",fontsize=14];
goal -> call_20 [label="calls"];
call_5042 [label="List.length\l(List.tl (Destruct(Sensor, 0, Option.get …)).Sensor_msgs.laserScan_ranges)",shape=box,style=filled,color="yellow",fontname="courier",fontsize=14];
expr_5037 -> call_5042 [label="calls"];
}
 Load graph Loading.. proof attempt summary full graph ground_instances 3 definitions 0 inductions 0 search_time 0.015s details Expand smt_stats (:added-eqs 784
 :arith-add-rows 6
 :arith-assert-lower 18
 :arith-assert-upper 18
 :arith-bound-prop 1
 :arith-eq-adapter 10
 :arith

Let's also verify that if we are receiving a `rosgraph_msgs/Clock` message, and the `min_range` is less than 0.5 meters ( scaled to 50000 in our message converter ), then there is an outgoing `geometry_msgs/Twist` message with `linear.x` velocity that is less or equal to zero: 

In [ ]:
verify ( fun state ->
  (  is_laser_scan state.incoming
  && 
  ) ==> no_positive_outgoing_velocity (one_step state)
)

In [32]:
let is_clock msg = 
  match msg with  Some ( Clock _ ) -> true | _ -> false ;;
  
let no_positive_outgoing_velocity state =
  let open Geometry_msgs in
  match state.outgoing with 
  | Some (Twist twist) -> twist.twist_linear.vector3_x <= 0
  | _ -> false ;;

val is_clock : incoming_msg option -> bool = <fun>
val no_positive_outgoing_velocity : state -> bool = <fun>


In [35]:
verify ( fun state ->
  (  is_clock state.incoming
  && match state.min_range with None -> false | Some m -> m < 50000 
  ) ==> no_positive_outgoing_velocity (one_step state)
)

- : state -> bool = <fun>
module CX : sig val state : state end


Refuted call graph proof-attempt digraph "call graph" {
goal [label="not\l(not\l (((Is_a(Clock, Option.get state.incoming) && Is_a(Some, state.incoming))\l && not (state.min_range = None))\l && not (50000 \<= Option.get state.min_range))\l \|\| (Is_a(Twist,\l Option.get (if state.incoming = None then state else …).outgoing)\l && Is_a(Some, (if state.incoming = None then state else …).outgoing))\l && (Destruct(Twist, 0,\l Option.get\l (if state.incoming = None then state else …).outgoing)).Geometry_msgs.twist_linear.Geometry_msgs.vector3_x\l \<= 0)",shape=box,style=filled,color="cyan",fontname="courier",fontsize=14];
call_3696 [label="recfun.fold_left_i.scan.1 anon_fun.get_min_and_direction.0\l(Destruct(Sensor, 0, Option.get …)).Sensor_msgs.laserScan_ranges (0, 0) 0",shape=box,style=filled,color="green",fontname="courier",fontsize=14];
call_3696 -> expr_4476 [label="="];
expr_4476 [label="if (Destruct(Sensor, 0, Option.get …)).Sensor_msgs.laserScan_ranges \<\> []\lthen\l recfun.fold_left_i.scan.1 anon_fun.get_min_and_direction.0\l (List.tl (Destruct(Sensor, 0, Option.get …)).Sensor_msgs.laserScan_ranges)\l (if 0 \<=\l List.hd\l (Destruct(Sensor, 0, Option.get …)).Sensor_msgs.laserScan_ranges\l then (0, 0)\l else\l (List.hd\l (Destruct(Sensor, 0, Option.get …)).Sensor_msgs.laserScan_ranges, 0))\l (1 + 0)\lelse (0, 0)",shape=box,style=filled,color="grey",fontname="courier",fontsize=14];
goal -> call_3696 [label="calls"];
call_4458 [label="List.length\l(List.tl (Destruct(Sensor, 0, Option.get …)).Sensor_msgs.laserScan_ranges)",shape=box,style=filled,color="yellow",fontname="courier",fontsize=14];
expr_4459 -> call_4458 [label="calls"];
call_4475 [label="recfun.fold_left_i.scan.1 anon_fun.get_min_and_direction.0\l(List.tl (Destruct(Sensor, 0, Option.get …)).Sensor_msgs.laserScan_ranges)\l(if 0 \<=\l List.hd\l (Destruct(Sensor, 0, Option.get …)).Sensor_msgs.laserScan_ranges\l then (0, 0)\l else\l (List.hd\l (Destruct(Sensor, 0, Option.get …)).Sensor_msgs.laserScan_ranges, 0))\l(1 + 0)",shape=box,style=filled,color="yellow",fontname="courier",fontsize=14];
expr_4476 -> call_4475 [label="calls"];
call_3695 [label="List.length\l(Destruct(Sensor, 0, Option.get …)).Sensor_msgs.laserScan_ranges",shape=box,style=filled,color="green",fontname="courier",fontsize=14];
call_3695 -> expr_4459 [label="="];
expr_4459 [label="if (Destruct(Sensor, 0, Option.get ….incoming)).Sensor_msgs.laserScan_ranges\l = []\lthen 0\lelse\l 1\l + List.length\l (List.tl\l (Destruct(Sensor, 0, Option.get ….incoming)).Sensor_msgs.laserScan_ranges)",shape=box,style=filled,color="grey",fontname="courier",fontsize=14];
goal -> call_3695 [label="calls"];
}
 Load graph Loading.. proof attempt summary full graph ground_instances 2 definitions 0 inductions 0 search_time 0.018s details Expand smt_stats (:added-eqs 1072
 :arith-assert-lower 22
 :arith-assert-upper 19
 :arith-bound-prop 1
 :arith-eq-adapter 14
 :arith-fixed-eqs 1
 :arith-offset-eqs 3
 :conflicts 19
 :datatype-accessor-ax 55
 :datatype-constructor-ax 208
 :datatype-occurs-check 1061
 :datatype-splits 38
 :decisions 189
 :del-clause 28
 :final-checks 8
 :max-memory 21.08
 :memory 21.08
 :mk-bool-var 450
 :mk-clause 68
 :num-allocs 637979847
 :num-checks 5
 :propagations 133
 :rlimit-count 81988) Expand start[0.018s]
 (if Is_a(Clock, Option.get :var_0:.incoming)
 && Is_a(Some, :var_0:.incoming)
 then true else false)
 && (if :var_0:.min_range = None then false
 else Option.get :var_0:.min_range < 50000)
 ==> (if Is_a(Twist,
 Option.get
 (if :var_0:.incoming = None then :var_0: else …).outgoing)
 && Is_a(Some,
 (if :var_0:.incoming = None then :var_0: else …).outgoing)
 then
 (Destruct(Twist, 0,
 Option.get
 (if :var_0:.incoming = None then :var_0: else …).outgoing)).Geometry_msgs.twist_linear.Geometry_msgs.vector3_x
 <= 0
 else false) simplify into not
(((Is_a(Clock, Option.get :var_0:.incoming) && Is_a(Some, :var_0:.incoming))
 && not (:var_0:.min_range = None))
 && not (50000 <= Option.get :var_0:.min_range))
|

In [36]:
one_step CX.state

- : state =
{mode = Driving; min_range = Some 48763; direction = None; incoming = None;
 outgoing =
  Some
   (Twist
     {Geometry_msgs.twist_linear =
       {Geometry_msgs.vector3_x = 10000; vector3_y = 0; vector3_z = 0};
      twist_angular =
       {Geometry_msgs.vector3_x = 0; vector3_y = 0; vector3_z = 0}})}
